# Pyspark Assignment  {TASK-3}
# (by Garvit)

#### References:
http://infohost.nmt.edu/tcc/help/pubs/pylxml/web/Element-getiterator.html

https://intellipaat.com/tutorial/spark-tutorial/working-with-keyvalue-pairs/

In [232]:
import findspark
findspark.init()

import xmltodict
import pprint
import pyspark.sql as sql
ss = sql.SparkSession.builder.appName('spark assignment').getOrCreate()
sc = ss.sparkContext

## XML (Tree based) Parsing with Python

In [4]:
import xml.etree.ElementTree as ET
F1_tree = ET.parse('DS1.xml')
F2_tree = ET.parse('DS2.xml')
root1 = F1_tree.getroot()
root2 = F2_tree.getroot()

##### Function for Parse all data from XML file

In [5]:
def parser(tree):
    data = []
    for i in tree.getiterator():
        if i.text.strip()!='':
                data.append((i.tag,i.text))
    return data

aa = parser(F1_tree)
bb = parser(F2_tree)

print('FILE-1 Raw Data','\n')
print(aa)
print('\n','----------------------------------','\n')
print('FILE-2 Raw Data','\n')
print(bb)

FILE-1 Raw Data 

[('fullname', 'raymond_Tschui'), ('city', 'New York'), ('state', 'NY'), ('country', 'US'), ('degree', "Bachelor's Degree"), ('major', 'Hotel administration'), ('fullname', 'raymond_Tschui'), ('city', 'New York'), ('state', 'NY'), ('country', 'US'), ('degree', "Master's Degree"), ('major', 'Hotel administration'), ('fullname', 'vinchi_Bolt'), ('city', 'Boston'), ('state', 'Massachusetts'), ('country', 'US'), ('degree', "Bachelor's Degree"), ('major', 'Hotel administration'), ('fullname', 'vinchi_Bolt'), ('city', 'Boston'), ('state', 'Massachusetts'), ('country', 'CAN'), ('degree', "Master's Degree"), ('major', 'Hotel administration')]

 ---------------------------------- 

FILE-2 Raw Data 

[('givenname', 'Raymond'), ('surname', 'Tschui'), ('title', 'Front Desk Assistant'), ('employer', 'Hilton Hotel Corporation'), ('description', 'Working as a front desk Assistant'), ('Salary', '50,000 USD'), ('title', 'Front Desk Assistant '), ('employer', 'Taj Hotel Corporation'), (

##### Function to convert Raw data into list of tuples

In [6]:
def list_of_tuple(root):
    L=[]
    for i in root:
        if i.getchildren():
            L.append(parser(i))
    return (L)
        
aa1 = list_of_tuple(root1)
bb1 = list_of_tuple(root2)

print('FILE-1 Data in list of tuple','\n')
print(aa1)
print('\n','----------------------------------','\n')
print('FILE-2 Data in list of tuple','\n')
print(bb1)

FILE-1 Data in list of tuple 

[[('fullname', 'raymond_Tschui'), ('city', 'New York'), ('state', 'NY'), ('country', 'US'), ('degree', "Bachelor's Degree"), ('major', 'Hotel administration')], [('fullname', 'raymond_Tschui'), ('city', 'New York'), ('state', 'NY'), ('country', 'US'), ('degree', "Master's Degree"), ('major', 'Hotel administration')], [('fullname', 'vinchi_Bolt'), ('city', 'Boston'), ('state', 'Massachusetts'), ('country', 'US'), ('degree', "Bachelor's Degree"), ('major', 'Hotel administration')], [('fullname', 'vinchi_Bolt'), ('city', 'Boston'), ('state', 'Massachusetts'), ('country', 'CAN'), ('degree', "Master's Degree"), ('major', 'Hotel administration')]]

 ---------------------------------- 

FILE-2 Data in list of tuple 

[[('givenname', 'Raymond'), ('surname', 'Tschui'), ('title', 'Front Desk Assistant'), ('employer', 'Hilton Hotel Corporation'), ('description', 'Working as a front desk Assistant'), ('Salary', '50,000 USD'), ('title', 'Front Desk Assistant '), ('emp

/home/garvit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  after removing the cwd from sys.path.


#### Converting data into RDD and Paired RDD

In [8]:
rdd_A = sc.parallelize(aa1)
rdd_B = sc.parallelize(bb1)

In [23]:
rdd_A1 = rdd_A.map(lambda x:(x[0][1].split('_')[0],x))
rdd_A1.collect()

[('raymond',
  [('fullname', 'raymond_Tschui'),
   ('city', 'New York'),
   ('state', 'NY'),
   ('country', 'US'),
   ('degree', "Bachelor's Degree"),
   ('major', 'Hotel administration')]),
 ('raymond',
  [('fullname', 'raymond_Tschui'),
   ('city', 'New York'),
   ('state', 'NY'),
   ('country', 'US'),
   ('degree', "Master's Degree"),
   ('major', 'Hotel administration')]),
 ('vinchi',
  [('fullname', 'vinchi_Bolt'),
   ('city', 'Boston'),
   ('state', 'Massachusetts'),
   ('country', 'US'),
   ('degree', "Bachelor's Degree"),
   ('major', 'Hotel administration')]),
 ('vinchi',
  [('fullname', 'vinchi_Bolt'),
   ('city', 'Boston'),
   ('state', 'Massachusetts'),
   ('country', 'CAN'),
   ('degree', "Master's Degree"),
   ('major', 'Hotel administration')])]

In [10]:
rdd_B1 = rdd_B.map(lambda x:(x[0][1].lower(),x))
rdd_B1.collect()

[('raymond',
  [('givenname', 'Raymond'),
   ('surname', 'Tschui'),
   ('title', 'Front Desk Assistant'),
   ('employer', 'Hilton Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '50,000 USD'),
   ('title', 'Front Desk Assistant '),
   ('employer', 'Taj Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '100,000 USD'),
   ('degree', "Bachelor's Degree"),
   ('major', 'Hotel administration')]),
 ('raymond',
  [('givenname', 'Raymond'),
   ('surname', 'Tschui'),
   ('title', 'Front Desk Assistant'),
   ('employer', 'Hilton Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '150,000 USD'),
   ('title', 'Front Desk Assistant '),
   ('employer', 'Taj Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '200,000 USD'),
   ('degree', "Master's Degree"),
   ('major', 'Hotel administration')]),
 ('vinchi',
  [('givenname', 'Vinchi'),

#### Joining RDD_A1 and RDD_B1

In [20]:
join_rdd = rdd_B1.join(rdd_A1)

In [85]:
join_rdd.collect()

[('raymond',
  ([('givenname', 'Raymond'),
    ('surname', 'Tschui'),
    ('title', 'Front Desk Assistant'),
    ('employer', 'Hilton Hotel Corporation'),
    ('description', 'Working as a front desk Assistant'),
    ('Salary', '50,000 USD'),
    ('title', 'Front Desk Assistant '),
    ('employer', 'Taj Hotel Corporation'),
    ('description', 'Working as a front desk Assistant'),
    ('Salary', '100,000 USD'),
    ('degree', "Bachelor's Degree"),
    ('major', 'Hotel administration')],
   [('fullname', 'raymond_Tschui'),
    ('city', 'New York'),
    ('state', 'NY'),
    ('country', 'US'),
    ('degree', "Bachelor's Degree"),
    ('major', 'Hotel administration')])),
 ('raymond',
  ([('givenname', 'Raymond'),
    ('surname', 'Tschui'),
    ('title', 'Front Desk Assistant'),
    ('employer', 'Hilton Hotel Corporation'),
    ('description', 'Working as a front desk Assistant'),
    ('Salary', '50,000 USD'),
    ('title', 'Front Desk Assistant '),
    ('employer', 'Taj Hotel Corporation'

In [288]:
join_rdd1 = join_rdd.map(lambda x:x[1][0])
join_rdd1.take(1)

[[('givenname', 'Raymond'),
  ('surname', 'Tschui'),
  ('title', 'Front Desk Assistant'),
  ('employer', 'Hilton Hotel Corporation'),
  ('description', 'Working as a front desk Assistant'),
  ('Salary', '50,000 USD'),
  ('title', 'Front Desk Assistant '),
  ('employer', 'Taj Hotel Corporation'),
  ('description', 'Working as a front desk Assistant'),
  ('Salary', '100,000 USD'),
  ('degree', "Bachelor's Degree"),
  ('major', 'Hotel administration')]]

In [289]:
join_rdd2 = join_rdd.map(lambda x:x[1])
join_rdd2.take(1)

[([('givenname', 'Raymond'),
   ('surname', 'Tschui'),
   ('title', 'Front Desk Assistant'),
   ('employer', 'Hilton Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '50,000 USD'),
   ('title', 'Front Desk Assistant '),
   ('employer', 'Taj Hotel Corporation'),
   ('description', 'Working as a front desk Assistant'),
   ('Salary', '100,000 USD'),
   ('degree', "Bachelor's Degree"),
   ('major', 'Hotel administration')],
  [('fullname', 'raymond_Tschui'),
   ('city', 'New York'),
   ('state', 'NY'),
   ('country', 'US'),
   ('degree', "Bachelor's Degree"),
   ('major', 'Hotel administration')])]

#### Fetching "only values" from paired RDD

In [350]:
data_converted = join_rdd2.map(lambda x: (x[1][0][1],x[1][1][1],x[1][2][1],x[1][3][1],x[1][4][1],x[1][5][1],x[0][0][1],x[0][1][1],x[0][2][1],x[0][3][1],x[0][4][1],x[0][5][1],x[0][6][1],x[0][7][1],x[0][8][1],x[0][9][1],x[0][10][1],x[0][11][1]))
data_converted.take(1)

[('raymond_Tschui',
  'New York',
  'NY',
  'US',
  "Bachelor's Degree",
  'Hotel administration',
  'Raymond',
  'Tschui',
  'Front Desk Assistant',
  'Hilton Hotel Corporation',
  'Working as a front desk Assistant',
  '50,000 USD',
  'Front Desk Assistant ',
  'Taj Hotel Corporation',
  'Working as a front desk Assistant',
  '100,000 USD',
  "Bachelor's Degree",
  'Hotel administration')]

#### Defining schema

In [351]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    
    StructField("fullname", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("country", StringType(), True),
    StructField("degree_F1", StringType(), True),
    StructField("major_F1", StringType(), True),
    StructField("givenname", StringType(), True),
    StructField("sirname", StringType(), True),
    StructField("title_1", StringType(), True),
    StructField("employer_1", StringType(), True),
    StructField("description_1", StringType(), True),
    StructField("salary_1", StringType(), True),
    StructField("title_2", StringType(), True),
    StructField("employer_2", StringType(), True),
    StructField("description_2", StringType(), True),
    StructField("salary_2", StringType(), True),
    StructField("degree_F2", StringType(), True),
    StructField("major_F2", StringType(), True)])
    

In [352]:
df = ss.createDataFrame(data_converted, schema)

In [353]:
df_pandas = df.toPandas()

In [354]:
df_pandas

,fullname,city,state,country,degree_F1,major_F1,givenname,sirname,title_1,employer_1,description_1,salary_1,title_2,employer_2,description_2,salary_2,degree_F2,major_F2
0,raymond_Tschui,New York,NY,US,Bachelor's Degree,Hotel administration,Raymond,Tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"50,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"100,000 USD",Bachelor's Degree,Hotel administration
1,raymond_Tschui,New York,NY,US,Master's Degree,Hotel administration,Raymond,Tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"50,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"100,000 USD",Bachelor's Degree,Hotel administration
2,raymond_Tschui,New York,NY,US,Bachelor's Degree,Hotel administration,Raymond,Tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"150,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"200,000 USD",Master's Degree,Hotel administration
3,raymond_Tschui,New York,NY,US,Master's Degree,Hotel administration,Raymond,Tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"150,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"200,000 USD",Master's Degree,Hotel administration
4,vinchi_Bolt,Boston,Massachusetts,US,Bachelor's Degree,Hotel administration,Vinchi,Bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"50,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"100,000 USD",Bachelor's Degree,Hotel administration
5,vinchi_Bolt,Boston,Massachusetts,CAN,Master's Degree,Hotel administration,Vinchi,Bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"50,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"100,000 USD",Bachelor's Degree,Hotel administration
6,vinchi_Bolt,Boston,Massachusetts,US,Bachelor's Degree,Hotel administration,Vinchi,Bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"120,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"180,000 USD",Master's Degree,Hotel administration
7,vinchi_Bolt,Boston,Massachusetts,CAN,Master's Degree,Hotel administration,Vinchi,Bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,"120,000 USD",Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,"180,000 USD",Master's Degree,Hotel administration


#### Names -> Lowercase

In [356]:
df_pandas['fullname'] = df_pandas['fullname'].str.lower()
df_pandas['givenname'] = df_pandas['givenname'].str.lower()
df_pandas['sirname'] = df_pandas['sirname'].str.lower()

#### Replacing unwanted character from salary

In [357]:
df_pandas['salary_1'] = df_pandas['salary_1'].apply(lambda x:x.replace('USD',''))
df_pandas['salary_1'] = df_pandas['salary_1'].apply(lambda x:x.replace(',',''))

df_pandas['salary_2'] = df_pandas['salary_2'].apply(lambda x:x.replace('USD',''))
df_pandas['salary_2'] = df_pandas['salary_2'].apply(lambda x:x.replace(',',''))

In [358]:
df_pandas

,fullname,city,state,country,degree_F1,major_F1,givenname,sirname,title_1,employer_1,description_1,salary_1,title_2,employer_2,description_2,salary_2,degree_F2,major_F2
0,raymond_tschui,New York,NY,US,Bachelor's Degree,Hotel administration,raymond,tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,50000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,100000,Bachelor's Degree,Hotel administration
1,raymond_tschui,New York,NY,US,Master's Degree,Hotel administration,raymond,tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,50000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,100000,Bachelor's Degree,Hotel administration
2,raymond_tschui,New York,NY,US,Bachelor's Degree,Hotel administration,raymond,tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,150000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,200000,Master's Degree,Hotel administration
3,raymond_tschui,New York,NY,US,Master's Degree,Hotel administration,raymond,tschui,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,150000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,200000,Master's Degree,Hotel administration
4,vinchi_bolt,Boston,Massachusetts,US,Bachelor's Degree,Hotel administration,vinchi,bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,50000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,100000,Bachelor's Degree,Hotel administration
5,vinchi_bolt,Boston,Massachusetts,CAN,Master's Degree,Hotel administration,vinchi,bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,50000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,100000,Bachelor's Degree,Hotel administration
6,vinchi_bolt,Boston,Massachusetts,US,Bachelor's Degree,Hotel administration,vinchi,bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,120000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,180000,Master's Degree,Hotel administration
7,vinchi_bolt,Boston,Massachusetts,CAN,Master's Degree,Hotel administration,vinchi,bolt,Front Desk Assistant,Hilton Hotel Corporation,Working as a front desk Assistant,120000,Front Desk Assistant,Taj Hotel Corporation,Working as a front desk Assistant,180000,Master's Degree,Hotel administration


#### Converting "salary" type from 'string' to 'integer'

In [359]:
df_pandas['salary_1'] = df_pandas['salary_1'].astype(int)
df_pandas['salary_2'] = df_pandas['salary_2'].astype(int)

#### converting Pandas DF to spark DF

In [ ]:
df2 = ss.createDataFrame(df_pandas)

#### registering "Dataframe" as temperory "Table"

In [376]:
sql.SQLContext(sparkSession=ss, sparkContext=ss.sparkContext).registerDataFrameAsTable(df2,'details')
# enabling backword compability by passing parameters in SQLContext()

### MIN Salary of both employee

In [446]:
df3 = sql.SQLContext(sparkContext=ss.sparkContext, sparkSession=ss) \
.sql('select * from (select fullname,city,state,country,salary_1, salary_2 from details \
     order by salary_1, salary_2) where fullname = "raymond_tschui" limit 1')

In [448]:
print("MIN salary of 'raymond_tschui'")
print()
df3.show()

MIN salary of 'raymond_tschui'

+--------------+--------+-----+-------+--------+--------+
|      fullname|    city|state|country|salary_1|salary_2|
+--------------+--------+-----+-------+--------+--------+
|raymond_tschui|New York|   NY|     US|   50000|  100000|
+--------------+--------+-----+-------+--------+--------+



In [451]:
df4 = sql.SQLContext(sparkContext=ss.sparkContext, sparkSession=ss) \
.sql('select * from (select fullname,city,state,country,salary_1, salary_2 from details \
     order by salary_1, salary_2) where fullname = "vinchi_bolt" limit 1')

In [453]:
print("MIN salary of 'vinchi_bolt'")
print()
df4.show()

MIN salary of 'vinchi_bolt'

+-----------+------+-------------+-------+--------+--------+
|   fullname|  city|        state|country|salary_1|salary_2|
+-----------+------+-------------+-------+--------+--------+
|vinchi_bolt|Boston|Massachusetts|     US|   50000|  100000|
+-----------+------+-------------+-------+--------+--------+



### MAX Salary of both employee

In [459]:
df5 = sql.SQLContext(sparkContext=ss.sparkContext, sparkSession=ss) \
.sql('select * from (select fullname,city,state,country,salary_1, salary_2 from details \
     order by salary_1 desc, salary_2 desc) where fullname = "raymond_tschui" limit 1')

In [460]:
print("MAX salary of 'raymond_tschui'")
print()
df5.show()

MAX salary of 'raymond_tschui'

+--------------+--------+-----+-------+--------+--------+
|      fullname|    city|state|country|salary_1|salary_2|
+--------------+--------+-----+-------+--------+--------+
|raymond_tschui|New York|   NY|     US|  150000|  200000|
+--------------+--------+-----+-------+--------+--------+



In [461]:
df6 = sql.SQLContext(sparkContext=ss.sparkContext, sparkSession=ss) \
.sql('select * from (select fullname,city,state,country,salary_1, salary_2 from details \
     order by salary_1 desc, salary_2 desc) where fullname = "vinchi_bolt" limit 1')

In [462]:
print("MAX salary of 'vinchi_bolt'")
print()
df6.show()

MAX salary of 'vinchi_bolt'

+-----------+------+-------------+-------+--------+--------+
|   fullname|  city|        state|country|salary_1|salary_2|
+-----------+------+-------------+-------+--------+--------+
|vinchi_bolt|Boston|Massachusetts|     US|  120000|  180000|
+-----------+------+-------------+-------+--------+--------+



In [464]:
df3_pandas = df3.toPandas() 
df4_pandas = df4.toPandas()
df5_pandas = df5.toPandas()
df6_pandas = df6.toPandas()

### Saving results in TSV Format

In [476]:
df3_pandas.to_csv('1.csv',sep='\t',header=True)
df4_pandas.to_csv('2.csv',sep='\t',header=True)
df5_pandas.to_csv('3.csv',sep='\t',header=True)
df6_pandas.to_csv('4.csv',sep='\t',header=True)